In [1]:
import pandas as pd
from core.config import GENERATED_DATA_FOLDER
from core.utils.file_path import solve_path

In [2]:
tmpo_desloc_file = solve_path('media_duracao_viagem_dist_a_dist.csv',
                             GENERATED_DATA_FOLDER)
tmpo_desloc = pd.read_csv(tmpo_desloc_file,
                         sep=';')

In [3]:
tmpo_desloc.head()

,ds_origem_nome,ds_destino_nome,duracao_media
0,AGUA RASA,AGUA RASA,7.918367
1,AGUA RASA,ARICANDUVA,25.000000
2,AGUA RASA,ARTUR ALVIM,30.000000
3,AGUA RASA,BARRA FUNDA,58.000000
4,AGUA RASA,BELA VISTA,30.000000


In [4]:
mask = tmpo_desloc['ds_destino_nome']!='Fora São Paulo'
#pegar soh dentro de SP
tmpo_desloc = tmpo_desloc[mask].copy()

In [5]:
empregos_file = solve_path('percent_empregos_por_dist.csv',
                          GENERATED_DATA_FOLDER)
empregos = pd.read_csv(empregos_file,
                      sep=';')

In [6]:
empregos.head()

,ds_nome,percent_empregos
0,AGUA RASA,0.007403
1,ALTO DE PINHEIROS,0.003344
2,ANHANGUERA,0.000587
3,ARICANDUVA,0.004772
4,ARTUR ALVIM,0.001394


In [7]:
mapeamento_empregos = dict(zip(
                        empregos['ds_nome'], empregos['percent_empregos']
))

In [8]:
tmpo_desloc['percent_empregos_destino'] = tmpo_desloc['ds_destino_nome'].map(mapeamento_empregos)

In [9]:
duracao_maior_uma_hora = tmpo_desloc['duracao_media']>60

In [10]:
tmpo_desloc[duracao_maior_uma_hora].sample(10)

,ds_origem_nome,ds_destino_nome,duracao_media,percent_empregos_destino
2175,MANDAQUI,SE,72.857143,0.053162
3257,SAPOPEMBA,ITAIM BIBI,122.500000,0.066347
3819,VILA MARIANA,CAPAO REDONDO,80.000000,0.002924
3205,SAO MIGUEL,VILA MARIANA,91.666667,0.022049
909,CIDADE DUTRA,PINHEIROS,67.000000,0.024524
517,CACHOEIRINHA,MOOCA,105.000000,0.010822
3219,SAO RAFAEL,IPIRANGA,120.000000,0.015832
328,BELEM,VILA SONIA,80.000000,0.006962
1110,CURSINO,SANTA CECILIA,75.000000,0.014162
207,BARRA FUNDA,SANTO AMARO,80.000000,0.047735


In [11]:
viagens_uma_hora = tmpo_desloc[~duracao_maior_uma_hora].copy()

In [14]:
percent_empregos_acessiveis = viagens_uma_hora.groupby('ds_origem_nome')[['percent_empregos_destino']].sum()

In [15]:
percent_empregos_acessiveis = percent_empregos_acessiveis.reset_index()

In [19]:
percent_empregos_acessiveis.sort_values(by='percent_empregos_destino').head()

,ds_origem_nome,percent_empregos_destino
55,PARELHEIROS,0.017006
24,CIDADE TIRADENTES,0.021845
41,JARDIM ANGELA,0.030520
46,LAJEADO,0.037632
61,PERUS,0.039127


In [21]:
percent_empregos_acessiveis.sort_values(by='percent_empregos_destino', ascending=False).head(10)

,ds_origem_nome,percent_empregos_destino
52,MOEMA,0.793116
33,ITAIM BIBI,0.788862
25,CONSOLACAO,0.783376
6,BELA VISTA,0.773844
60,PERDIZES,0.771138
91,VILA MARIANA,0.757271
43,JARDIM PAULISTA,0.756184
62,PINHEIROS,0.754315
14,CAMPO BELO,0.716971
89,VILA LEOPOLDINA,0.704515


In [22]:
fname = solve_path('percent_jobs_acessible.csv', GENERATED_DATA_FOLDER)
percent_empregos_acessiveis.to_csv(fname, index=False, sep=';')